In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from lxml import html
from bs4 import BeautifulSoup
import requests
import time
import re

In [12]:
def scrape_aritzia(url, wait_time=2):
    try:
        # Set Chrome options for headless mode
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36"
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument(f"user-agent={user_agent}")
        interactive_element_xpath = '//*[@id="primary"]/div[1]/div[1]/div[3]/div/ul/li[1]/a'
        loaded_content_xpath = '//*[@id="pdp-panel__details"]/div'

        # Initialize the WebDriver with headless mode
        driver = webdriver.Chrome(options=chrome_options)
        
        # Open the webpage
        driver.get(url)

        # Wait for the specified time before clicking the interactive element
        time.sleep(wait_time)  # Wait for the specified time in seconds

        # Find the interactive element
        interactive_element = driver.find_element(By.XPATH, interactive_element_xpath)
        
        # Click the interactive element
        interactive_element.click()

        # Wait for the loaded content to be visible
        loaded_element = WebDriverWait(driver, wait_time).until(
            EC.visibility_of_element_located((By.XPATH, loaded_content_xpath))
        )

        # Once loaded, scrape the content
        dynamic_content = loaded_element.text
        
        return dynamic_content
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None
        
    finally:
        # Close the WebDriver
        driver.quit()

# Example usage:
url = "https://www.aritzia.com/us/en/product/flor-top/115882013.html"

dynamic_content = scrape_aritzia(url)
if dynamic_content:
    print(dynamic_content)

Features
Fully fashioned construction means your knitwear fits you better with flat, smooth seams and less bulk. Plus, it reduces material waste in the production process.
Materials & Care
Content: 62% LENZING™ ECOVERO™ Viscose, 38% polyester; Ribbed Trim: 51% LENZING™ ECOVERO™ Viscose, 31% polyester, 15% nylon, 3% elastane
Care: Hand wash
Imported


In [ ]:
# test URLS
url = "https://www.aritzia.com/us/en/product/olive-mini-skirt/116002.html?dwvar_116002_color=28150" # has lenzing ecovero
url = "https://www.aritzia.com/us/en/product/flor-top/115882013.html" # top

In [57]:
import re
from selenium.webdriver.common.action_chains import ActionChains




def scrape_aritzia(url, wait_time=2):
    try:
        # Set Chrome options for headless mode
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36"
        user_agent_2 = "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36"
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument(f"user-agent={user_agent}")
        
        interactive_element_xpath = '//*[@id="primary"]/div[1]/div[1]/div[3]/div/ul/li[1]/a'
        loaded_content_xpath = '//*[@id="pdp-panel__details"]/div'

        # Initialize the WebDriver with headless mode
        driver = webdriver.Chrome(options=chrome_options)
        # Add mouse movements to simulate human-like behavior
        actions = ActionChains(driver)
        actions.move_by_offset(10, 10).perform()  # Move mouse by a small offset
        
        # Open the webpage
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        product_name_element = soup.find('h1', class_='js-product-detail__product-name')
        product_name = product_name_element.text.strip().title() if product_name_element else None  # Capitalize the first letter of every word

        # Wait for the specified time before clicking the interactive element
        time.sleep(wait_time)  # Wait for the specified time in seconds

        # Find the interactive element
        interactive_element = driver.find_element(By.XPATH, interactive_element_xpath)
        
        # Click the interactive element
        interactive_element.click()

        # Wait for the loaded content to be visible
        loaded_element = WebDriverWait(driver, wait_time).until(
            EC.visibility_of_element_located((By.XPATH, loaded_content_xpath))
        )

        # Once loaded, scrape the content
        dynamic_content = loaded_element.text
        
        # Extract materials using regular expressions
        materials_match = re.findall(r'(\d+)%\s*(.*?)\s*(?=,|\n|$)', dynamic_content)
        
        if materials_match:
            materials_dict = {"item": product_name}
            for percentage, fabric in materials_match:
                fabric = fabric.strip().lower().replace("™", "")  # Convert to lowercase and remove trademark symbol
                materials_dict[fabric] = int(percentage)
            
            # Create and return dictionary
            return materials_dict
        else:
            print("Materials information not found.")
            return None
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None
        
    finally:
        # Close the WebDriver
        driver.quit()

# Example usage:
url = "https://www.aritzia.com/us/en/product/the-%2780s-comfy-denim-shirt/120515.html?dwvar_120515_color=32984"


dynamic_content = scrape_aritzia(url)
if dynamic_content:
    print(dynamic_content)


{'item': "The '80S Comfy Denim Shirt", 'lyocell': 56, 'cotton': 29, 'linen': 15}


In [60]:
# if I want to randomly select the user agent
import random

# List of user agents
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36",
    "Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36"
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"
]

def get_random_user_agent():
    return random.choice(user_agents)

def scrape_aritzia(url, wait_time=2):
    try:
        # Set Chrome options for headless mode
        user_agent = get_random_user_agent()
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument(f"user-agent={get_random_user_agent()}")
        
        interactive_element_xpath = '//*[@id="primary"]/div[1]/div[1]/div[3]/div/ul/li[1]/a'
        loaded_content_xpath = '//*[@id="pdp-panel__details"]/div'

        # Initialize the WebDriver with headless mode
        driver = webdriver.Chrome(options=chrome_options)
        # Add mouse movements to simulate human-like behavior
        actions = ActionChains(driver)
        actions.move_by_offset(10, 10).perform()  # Move mouse by a small offset
        
        # Open the webpage
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        product_name_element = soup.find('h1', class_='js-product-detail__product-name')
        product_name = product_name_element.text.strip().title() if product_name_element else None  # Capitalize the first letter of every word

        # Wait for the specified time before clicking the interactive element
        time.sleep(wait_time)  # Wait for the specified time in seconds

        # Find the interactive element
        interactive_element = driver.find_element(By.XPATH, interactive_element_xpath)
        
        # Click the interactive element
        interactive_element.click()

        # Wait for the loaded content to be visible
        loaded_element = WebDriverWait(driver, wait_time).until(
            EC.visibility_of_element_located((By.XPATH, loaded_content_xpath))
        )

        # Once loaded, scrape the content
        dynamic_content = loaded_element.text
        
        # Extract materials using regular expressions
        materials_match = re.findall(r'(\d+)%\s*(.*?)\s*(?=,|\n|$)', dynamic_content)
        
        if materials_match:
            materials_dict = {"item": product_name}
            for percentage, fabric in materials_match:
                fabric = fabric.strip().lower().replace("™", "")  # Convert to lowercase and remove trademark symbol
                materials_dict[fabric] = int(percentage)
            
            # Create and return dictionary
            return materials_dict
        else:
            print("Materials information not found.")
            return None
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None
        
    finally:
        # Close the WebDriver
        driver.quit()

# Example usage:
url = "https://www.aritzia.com/us/en/product/olive-mini-skirt/116002.html?dwvar_116002_color=28150"

dynamic_content = scrape_aritzia(url)
if dynamic_content:
    print(dynamic_content)


{'item': 'Olive Mini Skirt', 'recycled polyester': 67, 'lenzing ecovero viscose': 29, 'elastane': 4}
